<a href="https://colab.research.google.com/github/kutayoncuyilmaz/Thesis/blob/main/SC_RoBERTa_IMDB_Final__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Sentiment Classification with RoBERTa Model on IMDB Dataset**


---




# Necessary Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Feb 26 12:20:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install sentencepiece transformers

In [ ]:
!pip install -q -U watermark

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

numpy       : 1.21.5
pandas      : 1.3.5
torch       : 1.10.0+cu111
transformers: 4.16.2



In [ ]:
import transformers
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams
from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Upload

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/df_2022-02-04.csv' , index_col=[0]) 
df.head()

,review,sentiment
33553,I really liked this Summerslam due to the look...,1
9427,Not many television shows appeal to quite as m...,1
199,The film quickly gets to a major chase scene w...,0
12447,Jane Austen would definitely approve of this o...,1
39489,Expectations were somewhat high for me when I ...,0


In [ ]:
df['sentiment'].value_counts()

0    12006
1    11994
Name: sentiment, dtype: int64

In [ ]:
class_names = ['negative', 'positive']

# Loading Tokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Dataset Class

In [ ]:
class ImdbDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt', truncation = True
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.5, random_state=101)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)

In [ ]:
df_train.shape, df_val.shape, df_test.shape

((12000, 2), (6000, 2), (6000, 2))

# Data Loader

In [ ]:
def loader_data (df, tokenizer, max_len, batch_size):
  ds = ImdbDataset(
    reviews=df.review.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
MAX_LEN = 512

In [ ]:
BATCH_SIZE = 4

loader_data_train = loader_data(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
loader_data_val = loader_data(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
loader_data_test = loader_data(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

# Loading Pre-trained RoBERTa Model


In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)
model = model.to(device)

# Hyperparameters for the Model



In [ ]:
EPOCHS = 5 #arbitrary

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(loader_data_train) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

# Training Step Function




In [ ]:
from sklearn import metrics
from tqdm import tqdm
def epoch_trainer(model, loader_data, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
    progress_loader = tqdm(loader_data)
    for d in progress_loader:
        input_ids = d["input_ids"].reshape(BATCH_SIZE,512).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        current_loss = loss.item()
        losses.append(current_loss)
        progress_loader.set_description(f"Current loss: {current_loss:6.4f}")
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

# Evaluation function for the model

In [ ]:
def model_evaluator(model, loader_data, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in loader_data:
            input_ids = d["input_ids"].reshape(4,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

# Fine Tuning of the Pre-trained Model

In [ ]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = epoch_trainer(
        model,
        loader_data_train,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = model_evaluator(
        model,
        loader_data_val, 
        device, 
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Datasets/roberta_imdb_final.bin')
        if val_acc-best_accuracy<=0.003 :
          break
        best_accuracy = val_acc
    else:
      break

Epoch 1/5
----------


Current loss: 0.8715: 100%|██████████| 3000/3000 [12:46<00:00,  3.91it/s]

Train loss 0.4131300511103085 Train accuracy 0.8986666666666666


Val loss 0.27878318445694944 Val accuracy 0.918

Epoch 2/5
----------


Current loss: 0.9312: 100%|██████████| 3000/3000 [12:49<00:00,  3.90it/s]

Train loss 0.21770029162650462 Train accuracy 0.9555833333333333


Val loss 0.3769166621664578 Val accuracy 0.93

Epoch 3/5
----------


Current loss: 0.0010: 100%|██████████| 3000/3000 [12:49<00:00,  3.90it/s]

Train loss 0.12086548246624686 Train accuracy 0.9781666666666666


Val loss 0.5167268967612957 Val accuracy 0.9223333333333333

CPU times: user 36min 10s, sys: 8min 37s, total: 44min 47s
Wall time: 44min 34s


# Performance Evalution

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Datasets/roberta_imdb_final.bin'))

<All keys matched successfully>

In [ ]:
model = model.to(device)

In [ ]:
test_acc, test_loss = model_evaluator(
  model,
  loader_data_test,
  device,
  len(df_test)
)

print('Test Accuracy :', test_acc)
print('Test Loss :', test_loss)

Test Accuracy : 0.9301666666666667
Test Loss : 0.3744013368381032


In [ ]:
def predictions_IMDB(model, loader_data):
    model = model.eval()
    
    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in loader_data:

            texts = d["review_text"]
            input_ids = d["input_ids"].reshape(4,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)

            loss = outputs[0]
            logits = outputs[1]
            
            _, preds = torch.max(outputs[1], dim=1)

            probs = F.softmax(outputs[1], dim=1)

            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = predictions_IMDB(
  model,
  loader_data_test
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names, digits=4))

              precision    recall  f1-score   support

    negative     0.9152    0.9469    0.9308      2976
    positive     0.9459    0.9137    0.9295      3024

    accuracy                         0.9302      6000
   macro avg     0.9306    0.9303    0.9302      6000
weighted avg     0.9307    0.9302    0.9302      6000



# Inference Time

In [ ]:
def predict_batch(model, batch, trial=10):
  with torch.no_grad():
    texts = batch["review_text"]
    input_ids = batch["input_ids"].reshape(-1,512).to(device)
    attention_mask = batch["attention_mask"].to(device)
    targets = batch["targets"].to(device)

    outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)

    loss = outputs[0]
    logits = outputs[1]

    _, preds = torch.max(outputs[1], dim=1)

In [ ]:
batch_size_trial = 32
curr_loader = iter(loader_data(df_test, tokenizer, MAX_LEN, batch_size_trial))



In [ ]:
%%timeit -n 10
predict_batch(model, next(curr_loader))


10 loops, best of 5: 504 ms per loop
